## Uploading a model

<div class="alert alert-block alert-info">
    &#9432; The code in this notebook can be executed <a href="https://www.opvious.io/notebooks/retro/notebooks/?path=guides/uploading-a-model.ipynb">directly from your browser</a>. You will need an Opvious account.
</div>

In this notebook we show how to upload a model such that it can be solved without access to the original specification. This approach is useful in automated environments and production, particularly in combination with version tags.

In [1]:
%pip install opvious

## Formulation

We will use the [bin-packing problem](https://www.opvious.io/notebooks/retro/notebooks/?path=examples/bin-packing.ipynb) as example and use the same formulation.

In [2]:
import opvious.modeling as om

class BinPacking(om.Model):
    """Bin-packing MIP formulation"""
    
    items = om.Dimension() # Set of items to be put into bins
    weight = om.Parameter.non_negative(items) # Weight of each item
    bins = om.interval(1, om.size(items), name="B") # Set of bins
    max_weight = om.Parameter.non_negative() # Maximum weight allowed in a bin
    assigned = om.Variable.indicator(bins, items, qualifiers=['bins']) # 1 if an item is assigned to a given bin, 0 otherwise
    used = om.Variable.indicator(bins) # 1 if a bin is used, 0 otherwise

    @om.constraint
    def each_item_is_assigned_once(self):
        """Constrains each item to be assigned to exactly one bin"""
        for i in self.items:
            yield om.total(self.assigned(b, i) for b in self.bins) == 1

    @om.constraint
    def bin_weights_are_below_max(self):
        """Constrains each bin's total weight to be below the maximum allowed"""
        for b in self.bins:
            bin_weight = om.total(self.weight(i) * self.assigned(b, i) for i in self.items)
            yield bin_weight <= self.used(b) * self.max_weight()

    @om.objective
    def minimize_bins_used(self):
        """Minimizes the total number of bins with at least one item"""
        return om.total(self.used(b) for b in self.bins)

model = BinPacking()

## Application

Instead of optimizing directly from the model as in the original notebook, we will first upload it. This requires two main pieces:

+ An API token, from which to generate an authenticated client.
+ A name for the uploaded formulation, which later be used to refer to it when optimizing.

In [3]:
import opvious

client = opvious.Client.from_environment(default_endpoint=opvious.DEMO_ENDPOINT)

FORMULATION_NAME = "bin-packing"

async def upload_model(version_tag=None):
    """Saves the bin-packing model so that it can be solved just from the formulation name

    Args:
        version_tag: Optional versioning tag used to target specific model versions.
    """
    await client.register_specification(model.specification(), FORMULATION_NAME, tag_names=[version_tag] if version_tag else None)

await upload_model()

We can now get solutions just with the formulation name. The code is very similar to the one in the original bin-packing example, we simply replaced the inline specification with a `FormulationSpecification`.

In [4]:
async def optimal_assignment(bin_max_weight, item_weights, version_tag=None):
    """Returns a grouping of items which minimizes the number of bins used
    
    Args:
        bin_max_weight: The maximum allowable total weight for all items assigned to a given bin
        item_weights: Mapping from item name to its (non-negative) weight
        version_tag: Model version tag
    """
    problem = opvious.Problem(
        specification=opvious.FormulationSpecification(FORMULATION_NAME), # Note the formulation reference
        parameters={'weight': item_weights, 'maxWeight': bin_max_weight},
    )
    solution = await client.solve(problem)
    assignment = solution.outputs.variable('assigned')
    return list(assignment.reset_index().groupby('bins')['items'].agg(tuple))

await optimal_assignment(15, {
    'light': 5,
    'medium': 10,
    'heavy': 15,
})

[('heavy',), ('light', 'medium')]

We can also make requests without the SDK: under the hood everything goes through the same API (see its OpenAPI specification [here](https://api.cloud.opvious.io/openapi.yaml)). To show how, we implement below a function which returns the minimum number of bins needed to fit the input items (our model's objective value) using the popular `requests` library.

In [5]:
%pip install requests

In [6]:
import requests

def minimum_bin_count(bin_max_weight, item_weights, version_tag='latest'):
    """Returns the minimum number of bins needed to fit the input items

    Args:
        bin_max_weight: The maximum allowable total weight for all items assigned to a given bin
        item_weights: Mapping from item name to its (non-negative) weight
        version_tag: Model version tag
    """
    response = requests.post(
        url='https://api.cloud.opvious.io/solve',
        headers={
            'accept': 'application/json',
            'authorization': f'Bearer {OPVIOUS_TOKEN}',
        },
        json={
            'problem': {
                'formulation': {'name': FORMULATION_NAME, 'specificationTagName': version_tag},
                'inputs': {
                    'parameters': [
                        {'label': 'maxWeight', 'entries': [{'key': [], 'value': bin_max_weight}]},
                        {'label': 'weight', 'entries': [{'key': k, 'value': v} for k, v in item_weights.items()]},
                    ]
                }
            }
        }
    )
    return response.json()['outcome']['objectiveValue']

if OPVIOUS_TOKEN:
    minimum_bin_count(15, {
        'light': 5,
        'medium': 10,
        'heavy': 15,
    })

2